In [1]:
import os
import shutil
import random
import json
from PIL import Image
import pandas as pd
from datetime import datetime, timedelta

import matplotlib.pyplot as plt

from bertopic import BERTopic

from bertopic.representation import KeyBERTInspired
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance

import sys
sys.path.insert(0, "../../")
sys.path.insert(0, "../../../hindsight/hindsight_server/")
from annotation_helpers import add_hindsight_frame_path, get_entity_image, visualize_annotations, get_entity_image
from annotations_db import HindsightAnnotationsDB

from db import HindsightDB
from utils import make_dir, ocr_results_to_str, add_datetimes

/Users/connorparish/miniconda3/envs/hindsight_server/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
annotations_db = HindsightAnnotationsDB()

In [3]:
annotations = annotations_db.get_annotations()

In [4]:
tweets_df = pd.read_csv("all_tweets-1727230360.csv")
tweets_df = add_datetimes(tweets_df)
tweets_df = tweets_df.dropna(subset=['tweet_text'])
# tweets_df = tweets_df.drop_duplicates(subset=['tweet_text'], keep="first")

In [5]:
extracted_tweet_ads_dir = "../../data/extracted_entities/tweets/ads/"
os.makedirs(extracted_tweet_ads_dir, exist_ok=True)

In [6]:
twitter_ads = tweets_df.loc[~tweets_df['ad_icon'].isnull()]

In [7]:
# parent_annotations = annotations.loc[~annotations['frame_id'].isnull()]
# id_to_frame_id = { i: int(f) for i, f in zip(parent_annotations['id'], parent_annotations['frame_id'])}
# twitter_ads['frame_id'] = twitter_ads['parent_annotations_id'].map(id_to_frame_id)

In [8]:
twitter_ads_annotations = annotations.loc[annotations['id'].isin(set(twitter_ads['parent_annotations_id']))]
twitter_ads_annotations['x2'] = twitter_ads_annotations['x'] + twitter_ads_annotations['w']
twitter_ads_annotations['y2'] = twitter_ads_annotations['y'] + twitter_ads_annotations['h']

In [9]:
for i, row in twitter_ads_annotations.iterrows():
    im = get_entity_image(row)
    ad_save_path = os.path.join(extracted_tweet_ads_dir, f"twitter_ad_{row['frame_id']}_{row['id']}.jpg")
    im.save(ad_save_path)